In [1]:
deployment = "gpt4"
model = "gpt-4"

In [2]:
import openai
def work_on(input):
    response = openai.ChatCompletion.create(
        engine=deployment, # 如果直接访问OpenAI GPT服务的同学，这里不要使用engine这个参数，要使用model，如： model=“gpt-4”
        model = model,
        messages=[
            {"role": "system", "content": "You are a senior software engineer."},   
            {"role": "user", "content": input}
        ],
        temperature = 0.9, 
        max_tokens = 1000,
      )
    return response.choices[0].message.content

In [3]:
print(work_on(
"""
为部署kubernetes nginx-deployment写一个HPA配置
集群的pod数在2到10之间，目标CPU 利用率60%
"""
))

这是一个简单的HPA配置样例，你可以根据你的需求进行修改：

```yaml
apiVersion: autoscaling/v2beta2
kind: HorizontalPodAutoscaler
metadata:
  name: nginx-deployment-hpa
  namespace: default
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: nginx-deployment
  minReplicas: 2
  maxReplicas: 10
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 60
```
这个配置会根据CPU利用率来自动调节nginx-deployment的Pod数量，当CPU利用率超过60%时，会自动增加Pod的数量（最多不超过10个）；当CPU利用率低于60%时，会自动减少Pod的数量（最少保留2个）。

这个HPA配置使用的是autoscaling/v2beta2版本的API，这个版本的API支持更多的度量标准类型和更灵活的目标指定，同时也更符合Kubernetes约定的API风格。


# Transformer Pipeline

In [9]:
!pip install --upgrade transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/13/30/54b59e73400df3de506ad8630284e9fd63f4b94f735423d55fc342181037/transformers-4.33.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 728.3 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for safetensors>=0.3.1 from https://files.pythonhosted.org/packages/34/0e/12d55d5dd648b8f7ea7216c5b7cef9703b4dbd3b2a042872c711d5e98551/safetensors-0.3.3-cp311-cp311-macosx_13_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.9/406.9 kB 3.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")

In [11]:
def caption(input):
    output = pipe(input)
    return output[0]['generated_text']

In [13]:
from IPython.display import Image
image_url = "https://free-images.com/md/7687/blue_jay_bird_nature.jpg"
#image_url = "/Users/mbj0593/Downloads/blue_jay_bird_nature.jpg"
display(Image(url=image_url)) 
print(caption(image_url))

araffy blue jay sitting on a tree stump in the winter


# 看图说话

In [16]:
import openai
def write_a_story(request):
    response = openai.ChatCompletion.create(
        engine=deployment, # 如果直接访问OpenAI GPT服务的同学，这里不要使用engine这个参数，要使用model，如： model=“gpt-4”
        temperature = 0,
        messages=[
            {"role": "system", "content": """
              You are a novel writer, please, write a small story (about 200 words) in Chinese according to user input,
              """,
            },
            {"role": "user", "content": request}
        ]
      )
    return response.choices[0].message.content

In [18]:
image_url = "https://free-images.com/sm/9e46/white_bengal_tiger_tiger_0.jpg"
display(Image(url=image_url)) 
print(write_a_story(caption(image_url)))

在动物园的一片绿色草坪上，躺着一只白虎。它的皮毛洁白如雪，眼睛闪烁着神秘的光芒。白虎慵懒地躺在草坪上，享受着阳光的沐浴，仿佛是一位高贵的君王。

白虎的身边，小朋友们兴奋地围观，他们的眼睛都被这只罕见的白虎吸引住了。他们纷纷拿出手机，拍下这一难得的景象。白虎似乎并不介意这些小朋友的围观，它依然悠然自得地躺在草坪上。

突然，白虎慢慢地抬起头，它的眼睛直视着远方。所有的小朋友都屏住了呼吸，他们惊奇地看着白虎。白虎似乎看到了什么，它突然从草坪上跃起，向远方奔去。

小朋友们惊讶地看着白虎的背影，他们都想知道白虎看到了什么。但是，他们只能看着白虎消失在远方，留下一片空荡荡的草坪和他们的惊奇。


# LangChain + HuggingFace

In [15]:
#import os
#os.environ['HUGGINGFACEHUB_API_TOKEN']='your key'

In [ ]:
from langchain import HuggingFaceHub, LLMChain
from langchain.prompts import PromptTemplate

llm = HuggingFaceHub(repo_id="bigscience/bloom", model_kwargs={"temperature": 0.5, "max_length": 64})
prompt = PromptTemplate(
    input_variables = ["hints"],
    template = "A story about the bird \n{hints}. "
)
chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
print(chain.run("araffy blue jay sitting on a tree stump in the winter"))